In [15]:
import pandas as pd
import re
import seaborn as sbn
import matplotlib.pyplot as plt
import emoji
import string
from emoticon_lib import replace_emoticons_with_text
from cleantext import clean
from deep_translator import (GoogleTranslator)
import html
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
import contractions
# import sys
# print(sys.executable)
# %pip install wordcloud
%pip install contractions


# Read data
df = pd.read_csv('cyberbullying_tweets.csv')
#show first 25 rows of data
# print(df.head(2).to_markdown())

Note: you may need to restart the kernel to use updated packages.


In [20]:
#EDA
# print(df.info())
# print(df.describe())
# print(df.isna().sum())
# print(df.columns.tolist())
# print(df.cyberbullying_type.unique())
# print(df.shape)
# print('Duplicated data num:', df.duplicated().sum())

# word cloud
# word_string=" ".join(df['tweet_text'].str.lower())
# plt.figure(figsize=(15,15))
# wordcloud = WordCloud(background_color="white", stopwords = STOPWORDS, max_words=50, max_font_size= 300,  width=1400, height=800).generate(word_string)

# plt.imshow(wordcloud, interpolation="bilinear")
# plt.axis('off')

In [21]:
#Remove duplication
# print(df.describe)
df =df.drop_duplicates(subset='tweet_text', keep="last")
 ############
#Change emojis and emoticon to text(in html code too)
df['tweet_text'] = df['tweet_text'].apply(lambda e: emoji.demojize(e, language="en"))

#Remove html tags
df['tweet_text'] = df['tweet_text'].apply(lambda e: html.unescape(e))

## Remove RT, #, URL and @:
df['tweet_text'] = df['tweet_text'].apply(lambda e: re.sub(r"(RT)|(:)|(@)|(?:\@|\#|https?|http?|www)\S+", "", e))

# Remove email 
df['tweet_text'] = df['tweet_text'].apply(lambda e: re.sub(r"\S+@\S+(?:\.\S+)+", "", e))

# Remove emoticons :/
df['tweet_text'] = df['tweet_text'].apply(lambda e: replace_emoticons_with_text(e))

# Translate
df['tweet_text'] = df['tweet_text'].apply(lambda e: GoogleTranslator(source='auto', target='en').translate(text = e))

# Excel
df.to_csv('new_cyberbully_tweet.csv', index=False)



#Remove stopwords






In [16]:
df1 = pd.read_csv('new_cyberbully_tweet.csv')
print(df1.isna().sum())
print(df1.info())

# remove new line, punctuation, lowercase all letters
df1['tweet_text'] = df1['tweet_text'].apply(lambda e: str(e).replace('\n',' ').lower())

print(df1.head(10).to_markdown())


# expand contractions
df1['tweet_text'] = df1['tweet_text'].apply(lambda e: contractions.fix(e))
print(df1.head(10).to_markdown())

# remove punctuations
def remove_punctuations(text):
    for char in string.punctuation:
        text = text.replace(char, '')
    return text
df1['tweet_text'] = df1['tweet_text'].apply(remove_punctuations)
print(df1.head(10).to_markdown())

# remove stopwords
stopword = stopwords.words('english')
df1['tweet_text'] = df1['tweet_text'].apply(lambda e: ' '.join([word for word in e.split() if word not in (stopword)]))
print(df1.head(10).to_markdown())

# tokenization
df1['tweet_text'] = df1.apply(lambda row: word_tokenize(row['tweet_text']), axis=1)
print(df1.head(10).to_markdown())

# # lemmatization
# pos_tag(word_tokenize('The most common method is word tokenizing.'))
# #stemming
# lancaster = SnowballStemmer()
# df['stemmed'] = df['tweet_text'].apply(lambda x: [lancaster.stem(t) for t in tokens]) # Stem every word.
# df = df.drop(columns=['unstemmed'])


tweet_text            185
cyberbullying_type      0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46017 entries, 0 to 46016
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          45832 non-null  object
 1   cyberbullying_type  46017 non-null  object
dtypes: object(2)
memory usage: 719.1+ KB
None
|    | tweet_text                                                                                                                        | cyberbullying_type   |
|---:|:----------------------------------------------------------------------------------------------------------------------------------|:---------------------|
|  0 | in other words  your food was crapilicious!                                                                                       | not_cyberbullying    |
|  1 | why is  so white?                                                                              